## Install library
Most common data and machine learning libraries are preinstalled such as Pandas, Sklearn, Pytorch, Tensorflow. Should you need additional libraries, this is how to do that

In [ ]:
#using networkx
!pip install --upgrade networkx

In [ ]:
!pip install dash dash-cytoscape


## Logon to Azure ML Workspace

In [2]:
from azureml.core import Workspace

ws = Workspace.from_config()


## Getting data from the workspace
Assuming that you have configured data store and dataset in Azure ML workspace, the following code allows you to read data from a registered dataset in Azure ML and convert to Pandas dataframe

### Option 1: programatically register datastore and  dataset


In [13]:
###Register datastore
from azureml.core import Datastore
from  azureml.data.dataset_factory import TabularDatasetFactory 
sas_token =''
datastore = Datastore.register_azure_blob_container(ws,account_name = 'adlsdatalakegen6',datastore_name='mltrainingsas2', container_name= 'mltraining', sas_token = sas_token )
###Create Dataset and register to workspace
dataset = TabularDatasetFactory.from_parquet_files(path=(datastore,"ISDWeather/year=2008/month=1/*.parquet"))

dataset = dataset.register(workspace=ws,
                                 name='idsdataset',
                                 description='IDS training data', create_new_version=True)


## Option 2:create datastore and dataset using AML UI


In [3]:
# then simply use this code to get the handle to the dataset
dataset = ws.datasets['isdweathertest']

In [4]:
#Get data in Pandas format 
idsdataset_pd = dataset.to_pandas_dataframe()

In [9]:
idsdataset_pd.shape


(8103070, 22)

## Visualizing Network

In [11]:
import plotly.graph_objects as go

import networkx as nx

G = nx.random_geometric_graph(200, 0.125)
edge_x = []
edge_y = []
for edge in G.edges():
    x0, y0 = G.nodes[edge[0]]['pos']
    x1, y1 = G.nodes[edge[1]]['pos']
    edge_x.append(x0)
    edge_x.append(x1)
    edge_x.append(None)
    edge_y.append(y0)
    edge_y.append(y1)
    edge_y.append(None)

edge_trace = go.Scatter(
    x=edge_x, y=edge_y,
    line=dict(width=0.5, color='#888'),
    hoverinfo='none',
    mode='lines')

node_x = []
node_y = []
for node in G.nodes():
    x, y = G.nodes[node]['pos']
    node_x.append(x)
    node_y.append(y)

node_trace = go.Scatter(
    x=node_x, y=node_y,
    mode='markers',
    hoverinfo='text',
    marker=dict(
        showscale=True,
        # colorscale options
        #'Greys' | 'YlGnBu' | 'Greens' | 'YlOrRd' | 'Bluered' | 'RdBu' |
        #'Reds' | 'Blues' | 'Picnic' | 'Rainbow' | 'Portland' | 'Jet' |
        #'Hot' | 'Blackbody' | 'Earth' | 'Electric' | 'Viridis' |
        colorscale='YlGnBu',
        reversescale=True,
        color=[],
        size=10,
        colorbar=dict(
            thickness=15,
            title='Node Connections',
            xanchor='left',
            titleside='right'
        ),
        line_width=2))
node_adjacencies = []
node_text = []
for node, adjacencies in enumerate(G.adjacency()):
    node_adjacencies.append(len(adjacencies[1]))
    node_text.append('# of connections: '+str(len(adjacencies[1])))

node_trace.marker.color = node_adjacencies
node_trace.text = node_text
fig = go.Figure(data=[edge_trace, node_trace],
             layout=go.Layout(
                title='<br>Network graph made with Python',
                titlefont_size=16,
                showlegend=False,
                hovermode='closest',
                margin=dict(b=20,l=5,r=5,t=40),
                annotations=[ dict(
                    text="Python code: <a href='https://plotly.com/ipython-notebooks/network-graphs/'> https://plotly.com/ipython-notebooks/network-graphs/</a>",
                    showarrow=False,
                    xref="paper", yref="paper",
                    x=0.005, y=-0.002 ) ],
                xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                yaxis=dict(showgrid=False, zeroline=False, showticklabels=False))
                )
fig.show()


##  Additional Resources
Machine Learning for Beginners: https://github.com/microsoft/ML-For-Beginners
Data Science for Begineers: https://github.com/microsoft/Data-Science-For-Beginners
Azure Machine Learning Sample Notebooks: https://github.com/Azure/MachineLearningNotebooks/